In [1]:
import pandas as pd

In [2]:
input_file = './youtube_data_newer.csv'
df = pd.read_csv(input_file)
df.dtypes

Unnamed: 0              int64
description            object
pub_date               object
title                  object
id                     object
caption                  bool
categoryId              int64
commentCount            int64
defaultLanguage        object
definition             object
dimension              object
dislikeCount            int64
duration               object
hasCustomThumbnail    float64
likeCount               int64
tags                   object
video_description      object
viewCount               int64
dtype: object

In [3]:
df_text = df[['title','description','tags','categoryId','defaultLanguage']]

In [4]:
df_text = df_text[df_text.tags != "[]"] #remove rows without tags
# df_text = df_text[df_text.defaultLanguage.startwith('en')]
df_text = df_text.loc[df_text['defaultLanguage'].str.startswith('en', na=False)]
# len(try_data)

In [5]:
print('length of data after removing empty rows: ', len(df_text))

length of data after removing empty rows:  2016


In [6]:
# df_text.dtypes
df_text['description'] = df_text['description'].astype('str')
df_text['title'] = df_text['title'].astype('str')

In [7]:
df_text['description'] = list(map(lambda x:x.lower(), df_text['description']))
df_text['title'] = list(map(lambda x:x.lower(), df_text['title']))

In [8]:
import re
def remove_url(x):
    result = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", x)
    return result

df_text['description'] = list(map(lambda x:remove_url(x), df_text['description']))
df_text['title'] = list(map(lambda x:remove_url(x), df_text['title']))

In [9]:
import html

df_text['description'] = list(map(lambda x:html.unescape(x),df_text['description']))
df_text['title'] = list(map(lambda x:html.unescape(x),df_text['title']))


In [10]:
APPOSTOPHES = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}


In [11]:
def con_trans(x):
    words = x.split()
    reformed = [APPOSTOPHES[word] if word in APPOSTOPHES else word for word in words]
    reformed = " ".join(reformed)
    return reformed
df_text['description'] = list(map(lambda x:con_trans(x),
                                              df_text['description']))
df_text['title'] = list(map(lambda x:con_trans(x),
                                              df_text['title']))

In [12]:
from nltk.corpus import stopwords
sw = stopwords.words("english")
newStopWords = ['subscribe','join','giveaway','instagram','follow','snap','twitter','channel']
sw.extend(newStopWords)
# print(sw)

In [13]:
def remove_stopword(words):
    out = []
    words = words.split()
    for word in words:
        word = word.lower()
        if word not in sw:
            out.append(word)
    return " ".join(out)
df_text['description'] = list(map(lambda x: remove_stopword(x), df_text['description']))
df_text['title'] = list(map(lambda x: remove_stopword(x), df_text['title']))

In [14]:
import string
def strip_punctuation(x):
    return x.translate(str.maketrans('', '', string.punctuation))
df_text['description']= list(map(lambda x: strip_punctuation(x), df_text['description']))
df_text['title']= list(map(lambda x: strip_punctuation(x), df_text['title']))
df_text['tags'] = list(map(lambda x:strip_punctuation(x), df_text['tags']))
df_text['tags'] = df_text['tags'].astype('str')


In [15]:
import nltk
# nltk.download('words')
words = set(nltk.corpus.words.words())

def remove_non_english_word(x):
    return " ".join(w for w in nltk.wordpunct_tokenize(x) if w.lower() in words or not w.isalpha())

df_text['description']= list(map(lambda x: remove_non_english_word(x), df_text['description']))
df_text['title']= list(map(lambda x: remove_non_english_word(x), df_text['title']))

In [16]:
df_text["title_and_description"] = df_text["title"].map(str) + " " + df_text["description"]
df_text['combined'] = df_text["title_and_description"].map(str) + " " + df_text['tags']

In [17]:
df_text['combined'] = df_text['combined'].str.replace('\d+', '')

In [18]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
df_text['combined'] = list(map(lambda x:deEmojify(x),
                                            df_text['combined']))

In [19]:
def remove_single_alphabet(words):
    out = []
    words = words.split()
    for word in words:
        if len(word)>2:
            out.append(word)
    return " ".join(out)
df_text['combined'] = list(map(lambda x:remove_single_alphabet(x),
                                            df_text['combined']))

In [20]:
from nltk.stem.snowball import SnowballStemmer
def stem(words):
    words = words.split()
#     print(words)
    out = []
    for word in words:
        word = SnowballStemmer("porter").stem(word)
#         print(word)
        out.append(word)
    return ' '.join(out)
df_text['after_stem'] = list(map(lambda x:stem(x),df_text['combined']))
    

In [21]:
df_text

,title,description,tags,categoryId,defaultLanguage,title_and_description,combined,after_stem
2,know doing right 🤨 2019,make sure enable push check merch store,best crashes crash fails crash and burn motorb...,23,en,know doing right 🤨 2019 make sure enable push ...,know doing right make sure enable push check m...,know do right make sure enabl push check merch...
4,range rover sport 2018 dragon challenge – extr...,new range rover sport 2018 record breaking fea...,extreme car climb car climb 2018 range rover s...,2,en,range rover sport 2018 dragon challenge – extr...,range rover sport dragon challenge extreme cli...,rang rover sport dragon challeng extrem climb ...
6,tutorial special,love much,makeup comedy,24,en,tutorial special love much,tutorial special love much makeup comedy,tutori special love much makeup comedi
14,history world every year,since 200000 humanity spread around globe huge...,History of the World Ollie Bye Map History Wor...,24,en-GB,history world every year since 200000 humanity...,history world every year since humanity spread...,histori world everi year sinc human spread aro...
21,sport s04e26 full episode,super cannot watch jack finally capture make e...,Oggy “Oggy and the Cockroaches” “Oggy et les C...,1,en,sport s04e26 full episode super cannot watch j...,sport full episode super cannot watch jack fin...,sport full episod super cannot watch jack fina...
23,top 5 love,watch wife try pepper world good time hey dona...,what chinese girls love what do chinese girls ...,22,en,top 5 love watch wife try pepper world good ti...,top love watch wife try pepper world good time...,top love watch wife tri pepper world good time...
26,12 stool health,understand really healthy easiest way it check...,health care health tips healthy lifestyle heal...,26,en,12 stool health understand really healthy easi...,stool health understand really healthy easiest...,stool health understand realli healthi easiest...
27,warren love life,democratic presidential to joe busy brushing y...,The Late Show Late Show Stephen Colbert Steven...,24,en,warren love life democratic presidential to jo...,warren love life democratic presidential joe b...,warren love life democrat presidenti joe busi ...
34,try food around world 10 try cut,content check sponsor series » subscribe,Watch Cut WatchCut video would you rather line...,24,en,try food around world 10 try cut content check...,try food around world try cut content check sp...,tri food around world tri cut content check sp...
35,c tutorial learn c c language,c language popular computer language used lang...,C Programming Language Programming Language So...,27,en,c tutorial learn c c language c language popul...,tutorial learn language language popular compu...,tutori learn languag languag popular comput la...


In [22]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from numpy.random import seed
seed(1)

counter = Counter(df_text['categoryId'].tolist())
top_10_varieties = {i[0]: idx for idx, i in enumerate(counter.most_common(10))}
# print(top_10_varieties)
df = df_text[df_text['categoryId'].map(lambda x: x in top_10_varieties)]
len(df)
# df

description_list = df['after_stem'].tolist()
varietal_list = [top_10_varieties[i] for i in df['categoryId'].tolist()]
varietal_list = np.array(varietal_list)

count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(description_list)


tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

train_x, test_x, train_y, test_y = train_test_split(x_train_tfidf, varietal_list, test_size=0.2)

clf = MultinomialNB().fit(train_x, train_y)
train_score = clf.predict(train_x)
y_score = clf.predict(test_x)
# print(y_score)
print(metrics.classification_report(test_y, y_score, digits=3))
# print(metrics.classification_report(train_y, train_score, digits=3))
# n_right = 0
# for i in range(len(y_score)):
#     if y_score[i] == test_y[i]:
#         n_right += 1

# print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

              precision    recall  f1-score   support

           0      0.524     0.926     0.669        94
           1      0.407     0.838     0.548        68
           2      0.857     0.353     0.500        34
           3      0.000     0.000     0.000        33
           4      1.000     0.273     0.429        22
           5      0.000     0.000     0.000        23
           6      1.000     0.625     0.769        24
           7      0.800     0.364     0.500        22
           8      1.000     0.125     0.222        16
           9      1.000     0.190     0.320        21

   micro avg      0.535     0.535     0.535       357
   macro avg      0.659     0.369     0.396       357
weighted avg      0.579     0.535     0.466       357



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [23]:
from sklearn.svm import SVC
seed(1)
clf = SVC(kernel='linear').fit(train_x, train_y)
y_score = clf.predict(test_x)
train_score = clf.predict(train_x)
# print(y_score)
print(metrics.classification_report(test_y, y_score, digits=3))
# print(metrics.classification_report(train_y, train_score, digits=3))


# n_right = 0
# for i in range(len(y_score)):
#     if y_score[i] == test_y[i]:
#         n_right += 1

# print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

              precision    recall  f1-score   support

           0      0.647     0.819     0.723        94
           1      0.557     0.794     0.655        68
           2      0.828     0.706     0.762        34
           3      0.722     0.394     0.510        33
           4      0.667     0.636     0.651        22
           5      0.833     0.217     0.345        23
           6      0.944     0.708     0.810        24
           7      0.812     0.591     0.684        22
           8      0.692     0.562     0.621        16
           9      0.750     0.714     0.732        21

   micro avg      0.675     0.675     0.675       357
   macro avg      0.745     0.614     0.649       357
weighted avg      0.705     0.675     0.665       357



In [24]:
from sklearn.ensemble import RandomForestClassifier
seed(1)
clf = RandomForestClassifier()
clf.fit(train_x, train_y)
y_score = clf.predict(test_x)
train_score = clf.predict(train_x)
# print(y_score)
print(metrics.classification_report(test_y, y_score, digits=3))
# print(metrics.classification_report(train_y, train_score, digits=3))


# n_right = 0
# for i in range(len(y_score)):
#     if y_score[i] == test_y[i]:
#         n_right += 1

# print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

              precision    recall  f1-score   support

           0      0.556     0.734     0.633        94
           1      0.500     0.691     0.580        68
           2      0.760     0.559     0.644        34
           3      0.583     0.212     0.311        33
           4      0.684     0.591     0.634        22
           5      0.571     0.174     0.267        23
           6      0.938     0.625     0.750        24
           7      0.773     0.773     0.773        22
           8      0.588     0.625     0.606        16
           9      0.810     0.810     0.810        21

   micro avg      0.611     0.611     0.611       357
   macro avg      0.676     0.579     0.601       357
weighted avg      0.632     0.611     0.596       357



/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
